<a href="https://colab.research.google.com/github/hman930/hman930/blob/main/crawling_notion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4 requests notion-client

In [ ]:
from bs4 import BeautifulSoup
import requests
from notion_client import Client

# Notion 설정
NOTION_API_KEY = '<본인 API>'
DATABASE_ID = '<본인 NOTION DATABASE>'
notion = Client(auth=NOTION_API_KEY)

def fetch_news(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    news_data = []
    for dt in soup.find_all('dt', class_='thumb'):
        a_tag = dt.find('a')
        if a_tag and 'href' in a_tag.attrs:
            title_tag = dt.find_next_sibling('dd', class_='articleSubject')
            title = title_tag.get_text(strip=True) if title_tag else "No Title"
            full_url = a_tag['href']
            news_data.append({
                'title': title,
                'url': full_url
            })
    return news_data

def add_to_notion(news_items):
    for item in news_items:
        notion.pages.create(
            parent={"database_id": DATABASE_ID},
            properties={
                'Title': {
                    'title': [{'text': {'content': item['title']}}]
                },
                'URL': {
                    'url': item['url']
                }
            }
        )

def main():
    url = "https://finance.naver.com/news/mainnews.naver"
    news_items = fetch_news(url)
    add_to_notion(news_items)
    print("뉴스 항목이 Notion 데이터베이스에 성공적으로 추가되었습니다.")

if __name__ == '__main__':
    main()


뉴스 항목이 Notion 데이터베이스에 성공적으로 추가되었습니다.
